In [1]:
import pandas as pd
import tarfile
from tqdm import tqdm
import json
import numpy as np
import time
from copy import deepcopy

In [ ]:
# Load data
zf = tarfile.open('/media/muzamil/Academia/Research/Data/Yelp/text/yelp_dataset')
zf.extractall()#('yelp_academic_dataset_review.json')

In [2]:
line_count = len(open("yelp_academic_dataset_review.json").readlines())
user_ids, business_ids, stars, dates, texts = [], [], [], [], []
with open("yelp_academic_dataset_review.json") as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        user_ids += [blob["user_id"]]
        business_ids += [blob["business_id"]]
        stars += [blob["stars"]]
        dates += [blob["date"]]
        texts += [blob["text"]]

ratings_ = pd.DataFrame(
    {"user_id": user_ids, 
     "business_id": business_ids, 
     "rating": stars, 
     "date": dates, 
     "text": texts
    }
)

100%|██████████| 8021122/8021122 [01:21<00:00, 98228.05it/s] 


In [3]:
user_counts = ratings_["user_id"].value_counts()
# Retain only active users (≥ 5 ratings)
active_users = user_counts.loc[user_counts >= 5]
ratings_ = ratings_.loc[ratings_.user_id.isin(active_users)]

In [4]:
ratings_.head()

,user_id,business_id,rating,date,text
1,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1.0,2013-12-07 03:16:52,I am actually horrified this place is still in...
2,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5.0,2015-12-05 03:18:11,I love Deagan's. I do. I really do. The atmosp...
3,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1.0,2011-05-27 05:30:52,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g..."
4,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4.0,2017-01-14 21:56:57,"Oh happy day, finally have a Canes near my cas..."
5,5vD2kmE25YBrbayKhykNxQ,nlxHRv1zXGT0c0K51q3jDg,5.0,2013-05-07 07:25:25,This is definitely my favorite fast food sub s...


In [5]:
n_users = len(ratings_.user_id.unique())
n_restaurants = len(ratings_.business_id.unique())
print('Unique Users: {0}, unique restaurants: {1}'.format(n_users, n_restaurants))

Unique Users: 343872, unique restaurants: 202565


In [6]:
# load user.json
line_count = len(open("yelp_academic_dataset_user.json").readlines())
users, names, review_counts, since, friends, useful, \
            funny, cool, n_fans, years_elite, average_stars = [], [], [], [], [], [], [], [], [], [], []
with open("yelp_academic_dataset_user.json") as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        users += [blob["user_id"]]
        names += [blob["name"]]
        review_counts += [blob["review_count"]]
        since += [blob["yelping_since"]]
        friends += [blob["friends"]]
        useful += [blob["useful"]]
        funny += [blob["funny"]]
        cool += [blob["cool"]]
        n_fans += [blob["fans"]]
        years_elite += [blob["elite"]]
        average_stars += [blob["average_stars"]]
        
users_ = pd.DataFrame(
    {"user_id": users, 
     "user_name": names,
     "user_review_count": review_counts,
     "user_yelp_since": since,
     "friends": friends,
     "useful_reviews": useful,
     "funny_reviews": funny,
     "cool_reviews": cool,
     "n_fans": n_fans,
     "years_elite": years_elite,
     "average_stars": average_stars
    }
)

100%|██████████| 1968703/1968703 [00:48<00:00, 40671.77it/s] 


In [7]:
# load business.json
line_count = len(open("yelp_academic_dataset_business.json").readlines())
business_ids, names, addresses, cities, states, latitudes, longitudes, stars, \
        review_counts, is_open, categories = [], [], [], [], [], [], [], [], [], [], []
with open("yelp_academic_dataset_business.json") as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        business_ids += [blob["business_id"]]
        names += [blob["name"]]
        addresses += [blob["address"]]
        cities += [blob["city"]]
        states += [blob["state"]]
        latitudes += [blob["latitude"]]
        longitudes += [blob["longitude"]]
        stars += [blob["stars"]]
        review_counts += [blob["review_count"]]
        is_open += [blob["is_open"]]
        categories += [blob["categories"]]
        
business_ = pd.DataFrame(
    {"business_id": business_ids, 
     "business_name": names,
     "business_address": addresses,
     "business_city": cities, 
     "business_state": states, 
     "business_latitude": latitudes, 
     "business_longitude": longitudes, 
     "stars": stars, 
     "review_counts": review_counts, 
     "is_open": is_open,
     "categories": categories}
)

100%|██████████| 209393/209393 [00:02<00:00, 97367.49it/s]


In [8]:
# only active users with [reviews > 5]
users_ = users_.loc[users_.user_id.isin(active_users)]

In [9]:
users_.head()

,user_id,user_name,user_review_count,user_yelp_since,friends,useful_reviews,funny_reviews,cool_reviews,n_fans,years_elite,average_stars
0,ntlvfPzc8eglqvk92iDIAw,Rafael,553,2007-07-06 03:27:11,"oeMvJh94PiGQnx_6GlndPQ, wm1z1PaJKvHgSDRKfwhfDg...",628,225,227,14,,3.57
1,FOBRPlBHa3WPHFB5qYDlVg,Michelle,564,2008-04-28 01:29:25,"ly7EnE8leJmyqyePVYFlug, pRlR63iDytsnnniPb3AOug...",790,316,400,27,"2008,2009,2010,2011,2012,2013",3.84
3,QaELAmRcDc5TfJEylaaP8g,John,206,2008-09-20 00:08:14,"iog3Nyg1i4jeumiTVG_BSA, M92xWY2Vr9w0xoH8bPplfQ...",233,160,84,6,2009,3.08
5,z5_82komKV3mI4ASGe2-FQ,Steve,186,2007-02-27 07:09:40,"E-fXXmALnZTPmzzL6rV4cQ, L2g6vS1GDfqLEpiDOFfCkA...",642,192,155,15,2007,2.88
7,f4_MRNHvN-yRn7EA8YWRxg,Jennifer,822,2011-01-17 00:18:23,"c-Dja5bexzEWBufNsHfRrQ, 02HJNyOzzYXvEKVApJb8GQ...",4127,2446,2878,137,"2011,2012,2013,2014,2015,2016,2017,2018",3.63


In [10]:
business_.head()

,business_id,business_name,business_address,business_city,business_state,business_latitude,business_longitude,stars,review_counts,is_open,categories
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,35.462724,-80.852612,3.5,36,1,"Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,33.569404,-111.890264,5.0,4,1,"Health & Medical, Fitness & Instruction, Yoga,..."
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,45.479984,-73.580070,5.0,5,1,"Pets, Pet Services, Pet Groomers"
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,36.219728,-115.127725,2.5,3,0,"Hardware Stores, Home Services, Building Suppl..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,33.428065,-111.726648,4.5,26,1,"Home Services, Plumbing, Electricians, Handyma..."


In [11]:
# Train Test Split
SAMPLING_RATE = 1/5

# Downsample by users
user_id_unique = ratings_.user_id.unique()
print(str(len(user_id_unique)))

user_id_sample = pd.DataFrame(user_id_unique, columns=['unique_user_id']) \
                    .sample(frac= SAMPLING_RATE, replace=False, random_state=1)
print(str(len(user_id_sample)))
ratings_sample = ratings_.merge(user_id_sample, left_on='user_id', right_on='unique_user_id') \
                    .drop(['unique_user_id'], axis=1)

# The following line is not applicable as we need random data samples, for that we need merge
# ratings_sample = ratings_.loc[ratings_.user_id.isin(user_id_sample)]

print(str(len(ratings_sample)))
print(ratings_sample.head())
print(ratings_sample.shape)

343872
68774
1081887
                  user_id             business_id  rating  \
0  jOERvhmK6_lo_XGUBPws_w  Ws8V970-mQt2X9CwCuT5zw     4.0   
1  jOERvhmK6_lo_XGUBPws_w  BqnxN4hHahZ-zZGXDDGLsg     4.0   
2  jOERvhmK6_lo_XGUBPws_w  tWiFat101ID5w_wgAPMXhA     3.0   
3  jOERvhmK6_lo_XGUBPws_w  P8IsTiHq5Hesa6UPL604ww     4.0   
4  jOERvhmK6_lo_XGUBPws_w  oi6mahpAmEuikfihFVDmCw     3.0   

                  date                                               text  
0  2009-10-13 04:16:41  I have been here twice. Very nice and laid bac...  
1  2009-11-11 04:38:31  We had a lovely dinner at BLT Steak for my bir...  
2  2009-09-10 04:38:08  *3.5 stars*\nWent to Ted's today for lunch. I ...  
3  2009-11-23 03:28:27  I don't know if Los Taquitos qualifies as a "d...  
4  2012-01-17 04:20:09  Good service, food was OK. I've tried the panc...  
(1081887, 5)


In [12]:
# hold out last review

# convert into pandas datetime
ratings_user_date = ratings_sample.loc[:, ['user_id', 'date']]
ratings_user_date.date = pd.to_datetime(ratings_user_date.date)

# Get the max date index value (most recent review)
index_holdout = ratings_user_date.groupby(['user_id'], sort=False)['date'].transform(max) == ratings_user_date['date']

# most recent review (Test set)
ratings_holdout_ = ratings_sample[index_holdout]
# All other reviews
ratings_traincv_ = ratings_sample[~index_holdout]

# Get the max date index value from 'ratings_traincv_' (2nd most recent review from original data)
ratings_user_date = ratings_traincv_.loc[:, ['user_id', 'date']]
index_holdout = ratings_user_date.groupby(['user_id'], sort=False)['date'].transform(max) == ratings_user_date['date']
# 2nd most recent review (Validation set)
ratings_cv_ = ratings_traincv_[index_holdout]
# Remaining dataset for Trainig
ratings_train_ = ratings_traincv_[~index_holdout]

In [13]:
ratings_train_.head()

,user_id,business_id,rating,date,text
0,jOERvhmK6_lo_XGUBPws_w,Ws8V970-mQt2X9CwCuT5zw,4.0,2009-10-13 04:16:41,I have been here twice. Very nice and laid bac...
1,jOERvhmK6_lo_XGUBPws_w,BqnxN4hHahZ-zZGXDDGLsg,4.0,2009-11-11 04:38:31,We had a lovely dinner at BLT Steak for my bir...
2,jOERvhmK6_lo_XGUBPws_w,tWiFat101ID5w_wgAPMXhA,3.0,2009-09-10 04:38:08,*3.5 stars*\nWent to Ted's today for lunch. I ...
3,jOERvhmK6_lo_XGUBPws_w,P8IsTiHq5Hesa6UPL604ww,4.0,2009-11-23 03:28:27,"I don't know if Los Taquitos qualifies as a ""d..."
4,jOERvhmK6_lo_XGUBPws_w,oi6mahpAmEuikfihFVDmCw,3.0,2012-01-17 04:20:09,"Good service, food was OK. I've tried the panc..."


In [14]:
# remove the user that has fake reviews

# remove rating records of Validation and Testing dataset from Training dataset.
# Delete Testing dataset from Training dataset.
holdout_users_del = set(ratings_holdout_.user_id) - set(ratings_train_.user_id)
ratings_holdout_ = ratings_holdout_[~ratings_holdout_.user_id.isin(holdout_users_del)]
# Delete Validation dataset from Training dataset.
cv_users_del = set(ratings_cv_.user_id) - set(ratings_train_.user_id)
ratings_cv_ = ratings_cv_[~ratings_cv_.user_id.isin(cv_users_del)]

# ratings_cv_ = ratings_cv_[~ratings_cv_.user_id.isin(['HiT9sg9pvDiEVMFHJYihXg'])]
# ratings_holdout_ = ratings_holdout_[~ratings_holdout_.user_id.isin(['HiT9sg9pvDiEVMFHJYihXg'])]

print('There are {0} rows, {1} columns in training set.'.format(ratings_train_.shape[0], ratings_train_.shape[1]))
print('There are {0} rows, {1} columns in validation set.'.format(ratings_cv_.shape[0], ratings_cv_.shape[1]))
print('There are {0} rows, {1} columns in holdout set.'.format(ratings_holdout_.shape[0], ratings_holdout_.shape[1]))

There are 944323 rows, 5 columns in training set.
There are 68779 rows, 5 columns in validation set.
There are 68774 rows, 5 columns in holdout set.


In [15]:
# check if we have a enough user sample size (> 50000)
number_of_unique_users = len(ratings_train_.user_id.unique())
print(number_of_unique_users)

68772


In [16]:
def process(df):
    # convert to pandas datetime format
    df['date']  = pd.to_datetime(df['date'])
    df['week_day'] = df['date'].dt.weekday
    df['month'] = df['date'].dt.month
    df['hour'] = df['date'].dt.hour
    df = df.merge(users_, on = 'user_id')
    df = df.merge(business_, on = 'business_id')
    rename_dict = {'business_longitude': 'longitude', 'business_latitude': 'latitude',
                  'business_state':'state','business_city':'city', 'business_address': 'address'}
    df = df.rename(columns = rename_dict)
    return df

ratings_train = process(ratings_train_.copy())
ratings_holdout = process(ratings_holdout_.copy())
ratings_val = process(ratings_cv_.copy())
print(str(len(ratings_train)))
print(str(len(ratings_holdout)))
print(str(len(ratings_val)))

944323
68774
68779


In [17]:
ratings_holdout.columns
ratings_holdout.head()

,user_id,business_id,rating,date,text,week_day,month,hour,user_name,user_review_count,...,business_name,address,city,state,latitude,longitude,stars,review_counts,is_open,categories
0,jOERvhmK6_lo_XGUBPws_w,bc4-cwjfQaCSDu3nh66Rkg,2.0,2019-08-18 01:14:05,"Beautiful, full of light store. The employees ...",6,8,1,Yazmin,112,...,L'Occitane,7014 E Camelback Rd,Scottsdale,AZ,33.503904,-111.930352,3.5,10,1,"Beauty & Spas, Shopping, Cosmetics & Beauty Su..."
1,4Jrv4wVyqBwNhKITSQDORg,bc4-cwjfQaCSDu3nh66Rkg,2.0,2019-09-20 07:13:33,My relationship with nearly everything L'Occit...,4,9,7,Melissa,18,...,L'Occitane,7014 E Camelback Rd,Scottsdale,AZ,33.503904,-111.930352,3.5,10,1,"Beauty & Spas, Shopping, Cosmetics & Beauty Su..."
2,HJECayULRM-6xh2GCCvLiA,d4qwVw4PcN-_2mK2o1Ro1g,5.0,2019-10-12 01:49:46,Really great Mexican fare here. Very busy plac...,5,10,1,Tim,314,...,Pink Taco,4455 Paradise Rd,Las Vegas,NV,36.109706,-115.154021,3.5,1120,1,"Mexican, Restaurants"
3,P_peDwsQ0bqCzqlqb0D_vA,d4qwVw4PcN-_2mK2o1Ro1g,4.0,2018-01-20 12:07:29,"We had lunch here this weekend, overall a dece...",5,1,12,Bradley,232,...,Pink Taco,4455 Paradise Rd,Las Vegas,NV,36.109706,-115.154021,3.5,1120,1,"Mexican, Restaurants"
4,WUvv4rFy_Z4SAXsBKR2jng,d4qwVw4PcN-_2mK2o1Ro1g,3.0,2013-02-06 06:55:12,The oh so famous Pink Taco.\n\nCame here for S...,2,2,6,Rachel,312,...,Pink Taco,4455 Paradise Rd,Las Vegas,NV,36.109706,-115.154021,3.5,1120,1,"Mexican, Restaurants"


In [18]:
# final data for training and validation
ratings_train_final = ratings_train.append(ratings_val)
# entire dataset for training, validation and testing
ratings_entire_df = ratings_train.append(ratings_val).append(ratings_holdout)

In [19]:
print(str(len(ratings_entire_df)))
ratings_entire_df.columns

1081876


Index(['user_id', 'business_id', 'rating', 'date', 'text', 'week_day', 'month',
       'hour', 'user_name', 'user_review_count', 'user_yelp_since', 'friends',
       'useful_reviews', 'funny_reviews', 'cool_reviews', 'n_fans',
       'years_elite', 'average_stars', 'business_name', 'address', 'city',
       'state', 'latitude', 'longitude', 'stars', 'review_counts', 'is_open',
       'categories'],
      dtype='object')

In [20]:
# One business per city
unique_city_businesses = ratings_entire_df[['city','business_id']].drop_duplicates()

# Cities must have at least 100 unique businesses
unique_cities = unique_city_businesses.groupby('city').count()['business_id']
print(str(len(unique_cities)))
unique_cities = unique_cities[unique_cities > 100]
print(str(len(unique_cities)))

# The users must also have made a postive review(above their historical average)
# to those selected restaurants
out = pd.DataFrame()
for city in unique_cities.index:
    # Only those business in city having rating greater than average rating
    tmp = ratings_holdout[(ratings_holdout['city'] == city) &
                              (ratings_holdout['rating'] > ratings_holdout['average_stars'])]
    # Sample only 5 users from each city
    if len(tmp['user_id'].unique()) > 4:
        ###this weird sampling technique is to ensure we dont' sample the same user twice in a same city
        five_users = np.random.choice(tmp['user_id'].unique(), 5, replace = False)
        row = tmp[tmp['user_id'].isin(five_users)].groupby('user_id', group_keys=False).apply(lambda df: df.sample(1))
        out = out.append(row)

# print(str(len(out)))
out.head()

944
95


,user_id,business_id,rating,date,text,week_day,month,hour,user_name,user_review_count,...,business_name,address,city,state,latitude,longitude,stars,review_counts,is_open,categories
26421,Oo7RsFJFOxlkHDp-K5atBA,sSWy-yEYPCHLss0LvmAhhA,4.0,2019-04-30 16:14:26,After my doctors appointment today I wanted so...,1,4,16,Kathy,22,...,Eggs Crepes Restaurant,250 Bayly Street W,Ajax,ON,43.842562,-79.027714,3.0,7,1,"Breakfast & Brunch, Restaurants, Creperies"
32748,Py2i0EJa7U7AYk4AUuKUAw,nx-7BYiMw878SWyBe5LDyA,4.0,2016-11-14 05:58:23,Had a Philly with poutine. Sandwich was good....,0,11,5,Thomas,14,...,Retro Burger,"105 Bayly Street W, Unit 18A",Ajax,ON,43.841955,-79.022369,4.0,36,1,"Burgers, Restaurants"
45659,Uly_rDkUppjM5jZ7mgUTfA,tD7_aIUTlfzyMoMkXXwiZA,5.0,2019-09-22 13:31:42,"First, let me start by saying we should all su...",6,9,13,Sylvia,12,...,Pizza Pino,855 Westney Road S,Ajax,ON,43.830514,-79.020845,4.0,101,1,"Restaurants, Pizza"
68219,iNto93QzBCGL7Fv4U3pHkg,QTxHD_-F4ljM-MxV_wTnUA,4.0,2012-01-21 18:05:08,"Great food! Not necessarily authentic, but ver...",5,1,18,Jeff,15,...,Mucho Burrito,"320 Harwood Avenue S, Unit 2",Ajax,ON,43.844276,-79.018902,3.0,19,1,"Mexican, Food, Restaurants"
67255,lDBkjaiazlp2EEcGK85OIQ,diL2jw9aYVoR8HfZdYVg0g,5.0,2018-09-09 18:34:14,I've been twice now and have thoroughly enjoye...,6,9,18,Adam,7,...,Firehouse Subs,170 Kingston Road E,Ajax,ON,43.864600,-79.016959,4.5,23,1,"Sandwiches, Restaurants, Salad, Fast Food, Delis"


In [21]:
predict_df = out[['user_id','city','state']]
predict_df = predict_df.merge(unique_city_businesses, on = 'city')
predict_df.to_csv('data/metric_sample.csv')

In [22]:
# random initialization, needs to be substituted by actual predictions later.
predict_df['predictions'] = 2.5

In [23]:
# How many of them are restaurants they have not visited (Serendipity)

def get_all_metrics(predict_df, validation_subsample, ratings_train_final):
    top_10_recs = predict_df.groupby(['user_id','city'])['predictions'].nlargest(10).reset_index()
    out = validation_subsample
    cnt =0
    serendipity = 0
    
    for row in out.iterrows():
        row_values = row[1]
#         print(row_values)
#         break
        top_10 = predict_df.loc[top_10_recs[top_10_recs['user_id'] == row_values['user_id']].level_2]['business_id']
        ###In top 10
        if row_values['business_id'] in top_10.values:
            cnt+=1
        user_history = ratings_train_final[ratings_train_final['user_id'] == row_values['user_id']]    
        been_there = [i for i in top_10.values if i in  user_history.business_id.values]
        serendipity += 1-len(been_there)/10
    
    top_10 = cnt/len(out)
    serendipity = serendipity/len(out)
    
    predict_df = predict_df.reset_index()
    
    analysis_df = predict_df.merge(top_10_recs, left_on = ['user_id','city','index'], \
                                   right_on = ['user_id','city','level_2'])
    
    coverage = (analysis_df.groupby('city')['business_id'].nunique()/50).values.mean()
    
    predict_df['rankings']=predict_df.groupby(['city','user_id'])['predictions']. \
                                                        rank(method="first",ascending = False)
    running_rankings =0
    for row in out.iterrows():
        row_values = row[1]
        user_recs = predict_df[(predict_df['user_id']==row_values['user_id'])
                            &(predict_df['city']==row_values['city'])
                             & (predict_df['business_id']==row_values['business_id'])
                              ]
#         assert len(user_recs)==1
        running_rankings += user_recs['rankings'].sum()

    avg_rank = running_rankings / len(out)
    print(top_10, coverage, serendipity, avg_rank)
    
    return top_10, coverage, serendipity, avg_rank

In [24]:
from surprise import SVD
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import GridSearchCV
from surprise import Dataset
from surprise import BaselineOnly

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

In [25]:
ratings_train = process(ratings_train_.copy())
ratings_test = process(ratings_holdout_.copy())
ratings_val = process(ratings_cv_.copy())
print(str(len(ratings_train)))
print(str(len(ratings_test)))
print(str(len(ratings_val)))

944323
68774
68779


In [26]:
ratings_train.columns
ratings_train.head()

,user_id,business_id,rating,date,text,week_day,month,hour,user_name,user_review_count,...,business_name,address,city,state,latitude,longitude,stars,review_counts,is_open,categories
0,jOERvhmK6_lo_XGUBPws_w,Ws8V970-mQt2X9CwCuT5zw,4.0,2009-10-13 04:16:41,I have been here twice. Very nice and laid bac...,1,10,4,Yazmin,112,...,The Urban Tea Loft,"11 W Boston St, Ste 2",Chandler,AZ,33.302013,-111.84192,4.0,126,0,"Home & Garden, Restaurants, American (New), Sh..."
1,4fNHV9nkjf84BmmKiP1Npw,Ws8V970-mQt2X9CwCuT5zw,4.0,2008-01-23 03:03:37,My mother and I tried this place out in Decemb...,2,1,3,Anne,113,...,The Urban Tea Loft,"11 W Boston St, Ste 2",Chandler,AZ,33.302013,-111.84192,4.0,126,0,"Home & Garden, Restaurants, American (New), Sh..."
2,2-XLTj0Cfd81_XlmHo02Aw,Ws8V970-mQt2X9CwCuT5zw,3.0,2011-07-31 18:06:28,This is my second visit to Urban Tea Loft and ...,6,7,18,Deb,1010,...,The Urban Tea Loft,"11 W Boston St, Ste 2",Chandler,AZ,33.302013,-111.84192,4.0,126,0,"Home & Garden, Restaurants, American (New), Sh..."
3,wxeTtPILiMl1L-yBB-D2HA,Ws8V970-mQt2X9CwCuT5zw,5.0,2010-11-23 20:54:54,Introduced another newbie to the Tea Loft. Sti...,1,11,20,Dani,233,...,The Urban Tea Loft,"11 W Boston St, Ste 2",Chandler,AZ,33.302013,-111.84192,4.0,126,0,"Home & Garden, Restaurants, American (New), Sh..."
4,wxeTtPILiMl1L-yBB-D2HA,Ws8V970-mQt2X9CwCuT5zw,5.0,2009-02-16 23:26:19,I had lunch here today. Took my Brownie troop ...,0,2,23,Dani,233,...,The Urban Tea Loft,"11 W Boston St, Ste 2",Chandler,AZ,33.302013,-111.84192,4.0,126,0,"Home & Garden, Restaurants, American (New), Sh..."


In [27]:
# remove observations that may cause cold-start problem, which breaks the model.
ratings_test = ratings_test.loc[ratings_test.business_id.isin(ratings_train.business_id)]
ratings_val = ratings_val.loc[ratings_val.business_id.isin(ratings_train.business_id)]
print(str(len(ratings_train)))
print(str(len(ratings_test)))
print(str(len(ratings_val)))

944323
63242
64220


In [28]:
trainset = ratings_train.loc[:,['user_id', 'business_id', 'rating']]
trainset.columns = ['userID', 'itemID','rating']
valset = ratings_val.loc[:, ['user_id', 'business_id', 'rating']]
valset.columns = ['userID', 'itemID','rating']
testset = ratings_holdout.loc[:, ['user_id', 'business_id', 'rating']]
testset.columns = ['userID', 'itemID','rating']

In [29]:
# The Reader class is used to parse a file containing ratings.
# Such a file is assumed to specify only one rating per line,
# and each line needs to respect the following structure:
# user ; item ; rating ; [timestamp]

reader = Reader(rating_scale = (0.0, 5.0))
train_data = Dataset.load_from_df(trainset[['userID','itemID','rating']], reader)
val_data = Dataset.load_from_df(valset[['userID','itemID','rating']], reader)
test_data = Dataset.load_from_df(testset[['userID','itemID','rating']], reader)

train_sr = train_data.build_full_trainset()
val_sr_before = val_data.build_full_trainset()
val_sr = val_sr_before.build_testset()
test_sr_before = test_data.build_full_trainset()
test_sr = test_sr_before.build_testset()

In [30]:
# You can configure the way baselines are computed using the
# bsl_options parameter passed at the creation of an algorithm. 
# This parameter is a dictionary for which the key 'method' indicates 
# the method to use. Accepted values are 'als' (default) and 'sgd'. 
# Depending on its value, other options may be set.

# Hyperparameters for Baseline method
# @method: optimization method
# @ als: Alternating Least Square: a method that approaches solution for 
#        our baseline model’s loss function (*) via iterative updates on 
#        both bᵤ and bᵢ alternatively.
bsl_options = {'method': 'als', 'n_epochs':3}
bias_baseline = BaselineOnly(bsl_options)
# The algo.fit() fits the model on training set.
bias_baseline.fit(train_sr)
# The algo.test() yields predictions on validation set.
predictions = bias_baseline.test(val_sr)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 1.3443


1.344325874634335

In [31]:
bsl_options = {'method': 'als', 'n_epochs':5}
bias_baseline = BaselineOnly(bsl_options)
bias_baseline.fit(train_sr)
predictions = bias_baseline.test(val_sr)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 1.3444


1.344353412579548

In [32]:
bsl_options = {'method': 'als', 'n_epochs':9}
bias_baseline = BaselineOnly(bsl_options)
bias_baseline.fit(train_sr)
predictions = bias_baseline.test(val_sr)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 1.3444


1.344354354759994

In [34]:
# Observation: 
# It seems different hyperparameters all performs the same result; 
# the team just uses default.

# Evaluation:
# Note: the team has evaluated this and the following algorithms 
# on 20%, 50%, 100% training data respectively. For readability, 
# only result on 20% data is displayed. For full result, please 
# refer to a result table enclosed.

In [33]:
# runtime on 20% data
start_time = time.time()
bias_baseline.fit(train_sr)
print("--- %s seconds ---" % (time.time() - start_time))

Estimating biases using als...
--- 2.0309667587280273 seconds ---


In [39]:
# Regression metrics
# The most commonly used metric for regression tasks is 
# RMSE (Root Mean Square Error). This is defined as the square root 
# of the average squared distance between the actual score and the predicted score:
# rmse=√∑ni=1(yi−yi^)2/n
# Here, yi denotes the true score for the i-th data point, and ŷ i$^ denotes 
# the predicted value. One intuitive way to understand this formula is that it is 
# the Euclidean distance between the vector of the true scores and the vector of 
# the predicted scores, averaged by √n, where n is the number of data points.

# R-squared: it ranges from zero to one, with zero indicating that the proposed model
# does not improve prediction over the mean model, and one indicating perfect prediction.

# RMSE is a good measure of how accurately the model predicts the response, 
# and it is the most important criterion for fit if the main purpose of 
# the model is prediction.

# Mean Absolute Error: MAE is the absolute difference between the target value and 
# the value predicted by the model. The MAE is more robust to outliers and does not penalize 
# the errors as extremely as mse. MAE is a linear score which means all the individual 
# differences are weighted equally. It is not suitable for applications where you want 
# to pay more attention to the outliers.

# *** Higher values of R^2 indicate better fit. ***
# *** Lower values of RMSE indicate better fit. ***
# *** Lower values of MAE indicate better fit. ***

bbase_p = bias_baseline.test(test_sr)
start_time = time.time()
bbase_20_df = pd.DataFrame(bbase_p, columns = ['userId','itemId','rating','pred_rating','x'])
bbase_20_df.head()
print(str(len(bbase_20_df)))
accuracy.rmse(bbase_p)
print('R^2 (with 20% data): ', r2_score(bbase_20_df.rating , bbase_20_df.pred_rating))
print('MAE (with 20% data): ', mean_absolute_error(bbase_20_df.rating, bbase_20_df.pred_rating))
print("--- %s seconds ---" % (time.time() - start_time))


68774
RMSE: 1.4082
R^2 (with 20% data):  0.17462442189058336
MAE (with 20% data):  1.1842901828893122
--- 0.09343266487121582 seconds ---


In [42]:
# Ranking + coverage + novelty metrics
predict_df_baseline = predict_df.copy()
eval_set = Dataset.load_from_df(predict_df_baseline[['user_id','business_id','predictions']], reader)

bias_basline = BaselineOnly({'method': 'als', 'n_epochs':9})
eval_before = eval_set.build_full_trainset()
eval_sr = eval_before.build_testset()
bias_basline.fit(train_sr)
eval_pred = bias_basline.test(eval_sr)
#accuracy.rmse(predictions_50)
baseline_20 = pd.DataFrame(eval_pred, columns = ['userId','itemId','rating','pred_rating','x'])
predict_df_baseline['predictions'] = baseline_20.pred_rating

Estimating biases using als...


In [51]:
predict_df_baseline_unique = predict_df_baseline.groupby('user_id')
predict_df_baseline_unique.head(50)
print(str(len(predict_df_baseline_unique)))

470


In [57]:
# print(str(len(out)))
# print(str(len(ratings_train_final)))
top_10_recs = predict_df.groupby(['user_id','city'])['predictions'].nlargest(10).reset_index()
print(str(len(top_10_recs)))
print(top_10_recs)

4700
                     user_id     city  level_2  predictions
0     --neBDssyZlHqAWgrHtUBQ  Vaughan   663375          2.5
1     --neBDssyZlHqAWgrHtUBQ  Vaughan   663376          2.5
2     --neBDssyZlHqAWgrHtUBQ  Vaughan   663377          2.5
3     --neBDssyZlHqAWgrHtUBQ  Vaughan   663378          2.5
4     --neBDssyZlHqAWgrHtUBQ  Vaughan   663379          2.5
...                      ...      ...      ...          ...
4695  zzo--VpSQh8PpsGVeMC1dQ  Vaughan   666196          2.5
4696  zzo--VpSQh8PpsGVeMC1dQ  Vaughan   666197          2.5
4697  zzo--VpSQh8PpsGVeMC1dQ  Vaughan   666198          2.5
4698  zzo--VpSQh8PpsGVeMC1dQ  Vaughan   666199          2.5
4699  zzo--VpSQh8PpsGVeMC1dQ  Vaughan   666200          2.5

[4700 rows x 4 columns]


In [52]:
top_10, coverage, serendipity, avg_rank = get_all_metrics(predict_df_baseline, out, ratings_train_final)

0.13617021276595745 0.20191489361702125 0.9834042553191483 538.5361702127659


In [39]:
import matplotlib.pyplot as plt
import json
from tqdm import tqdm

In [40]:
# This will follow the same process as the above model so can be skipped
def process(df):
    df['date']  = pd.to_datetime(df['date'])
    df['week_day'] = df['date'].dt.weekday
    df['month'] = df['date'].dt.month
    df['hour'] = df['date'].dt.hour
    df = df.merge(users_, on = 'user_id')
    df = df.merge(business_, on = 'business_id')
    rename_dict = {'business_longitude': 'longitude', 'business_latitude': 'latitude',
              'business_state':'state','business_city':'city', 'business_address': 'address'}
    df = df.rename(columns = rename_dict)
    return df
ratings_train = process(ratings_train_.copy())
ratings_test = process(ratings_holdout_.copy())
ratings_val = process(ratings_cv_.copy())

In [41]:
ratings_test = ratings_test.loc[ratings_test.business_id.isin(ratings_train.business_id)]
ratings_val = ratings_val.loc[ratings_val.business_id.isin(ratings_train.business_id)]

In [43]:
trainset = ratings_train.loc[:,['user_id', 'business_id', 'rating']]
trainset.columns = ['userID', 'itemID','rating']
valset = ratings_val.loc[:, ['user_id', 'business_id', 'rating']]
valset.columns = ['userID', 'itemID','rating']
testset = ratings_holdout.loc[:, ['user_id', 'business_id', 'rating']]
testset.columns = ['userID', 'itemID','rating']

In [44]:
reader = Reader(rating_scale = (0.0, 5.0))
train_data = Dataset.load_from_df(trainset[['userID','itemID','rating']], reader)
val_data = Dataset.load_from_df(valset[['userID','itemID','rating']], reader)
test_data = Dataset.load_from_df(testset[['userID','itemID','rating']], reader)

train_sr = train_data.build_full_trainset()
val_sr_before = val_data.build_full_trainset()
val_sr = val_sr_before.build_testset()
test_sr_before = test_data.build_full_trainset()
test_sr = test_sr_before.build_testset()

In [45]:
RMSE_tune = {}
n_epochs = [10, 20, 30]  # the number of iteration of the SGD procedure
lr_all = [0.001, 0.003, 0.005] # the learning rate for all parameters
reg_all =  [0.02, 0.05, 0.1, 0.4, 0.5] # the regularization term for all parameters

for n in n_epochs:
    for l in lr_all:
        for r in reg_all:
            print('Fitting n: {0}, l: {1}, r: {2}'.format(n, l, r))
            algo = SVD(n_epochs = n, lr_all = l, reg_all = r)
            algo.fit(train_sr)
            predictions = algo.test(val_sr)
            RMSE_tune[n,l,r] = accuracy.rmse(predictions)

Fitting n: 10, l: 0.001, r: 0.02
RMSE: 1.4361
Fitting n: 10, l: 0.001, r: 0.05
RMSE: 1.4360
Fitting n: 10, l: 0.001, r: 0.1
RMSE: 1.4365
Fitting n: 10, l: 0.001, r: 0.4
RMSE: 1.4377
Fitting n: 10, l: 0.001, r: 0.5
RMSE: 1.4385
Fitting n: 10, l: 0.003, r: 0.02
RMSE: 1.3826
Fitting n: 10, l: 0.003, r: 0.05
RMSE: 1.3835
Fitting n: 10, l: 0.003, r: 0.1
RMSE: 1.3836
Fitting n: 10, l: 0.003, r: 0.4
RMSE: 1.3875
Fitting n: 10, l: 0.003, r: 0.5
RMSE: 1.3894
Fitting n: 10, l: 0.005, r: 0.02
RMSE: 1.3569
Fitting n: 10, l: 0.005, r: 0.05
RMSE: 1.3561
Fitting n: 10, l: 0.005, r: 0.1
RMSE: 1.3568
Fitting n: 10, l: 0.005, r: 0.4
RMSE: 1.3628
Fitting n: 10, l: 0.005, r: 0.5
RMSE: 1.3647
Fitting n: 20, l: 0.001, r: 0.02
RMSE: 1.4038
Fitting n: 20, l: 0.001, r: 0.05
RMSE: 1.4043
Fitting n: 20, l: 0.001, r: 0.1
RMSE: 1.4048
Fitting n: 20, l: 0.001, r: 0.4
RMSE: 1.4081
Fitting n: 20, l: 0.001, r: 0.5
RMSE: 1.4095
Fitting n: 20, l: 0.003, r: 0.02
RMSE: 1.3485
Fitting n: 20, l: 0.003, r: 0.05
RMSE: 1.3478


In [46]:
import operator
min(RMSE_tune.items(), key=operator.itemgetter(1))[0]

(30, 0.005, 0.1)

In [47]:
# train and test on the optimal parameter
start_time = time.time()
algo_real = SVD(n_epochs = 30, lr_all = 0.005, reg_all = 0.1)
algo_real.fit(train_sr)
predictions = algo_real.test(test_sr)

In [48]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 90.21060490608215 seconds ---


In [49]:
accuracy.rmse(predictions)

RMSE: 1.3831


1.3831055486648447

In [50]:
accuracy.mae(predictions)

MAE:  1.1368


1.1367805351866709

In [51]:
r2_score([t[2] for t in predictions], [t[3] for t in predictions])

0.20381872413296298

In [52]:
# To evaluate coverage and serendipity metrics, use evaluation set created earlier.
predict_df_20 = pd.read_csv('data/metric_sample.csv', index_col=0)
predict_df_20['predictions'] = 2.5 # fill in this value temporally
eval_20 = Dataset.load_from_df(predict_df_20[['user_id','business_id','predictions']], reader)

In [53]:
eval_before_20 = eval_20.build_full_trainset()
eval_sr_20 = eval_before_20.build_testset()
eval_pred_20 = algo_real.test(eval_sr_20)

baseline_20 = pd.DataFrame(eval_pred_20, columns = ['userId','itemId','rating','pred_rating','x'])
predict_df_20['predictions'] = baseline_20.pred_rating

In [ ]:
top_10, coverage, serendipity, avg_rank = get_all_metrics(predict_df_20, out, ratings_train_final)